In [ ]:
%reload_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import math
import pickle

from matplotlib import pyplot as plt

from sass.utils.utils_data import ResultAnalysis

## Calcuate and visualize recall curve

In [ ]:
def plot_recall(top_n, true_scores, f_names, is_score: bool, titles: list = [], exp_product_ns = None, show_plot=True):
    
    if show_plot:
        n_col = 3
        n_row = math.ceil(len(f_names) / n_col)
        row_size = 3
        col_size = 4
        fig, axes = plt.subplots(n_row, n_col, figsize=[col_size*n_col, row_size*n_row])
    if not titles:
        titles = [None] * len(f_names)
    
    if not exp_product_ns:
        exp_product_ns = [None] * len(f_names)
        
    auc_vals = []
    for (i, f_name), p_n in zip(enumerate(f_names), exp_product_ns):
        if true_scores is None:
            with open(true_scores, 'rb') as f:
                _true_scores = pickle.load(f)
        else:
            _true_scores = true_scores

        if is_score:
            with open(f_name, 'rb') as f:
                scores = pickle.load(f)
            analyzer = ResultAnalysis(_true_scores, exp_scores=scores, sort_scores=False,
                                     max_exp_product_n=p_n,)
        else:
            analyzer = ResultAnalysis(_true_scores, exp_scores=None, 
                                      exp_product_file=f_name, sort_scores=False,
                                     max_exp_product_n=p_n,
                                     )
        analyzer.calc_recall_auc(top_n)
                
        if show_plot:
            title = titles[i]
            if title is None:
                title = f_name.split('_')[1:-2]
                title = '_'.join(title).split('/')[-1]
            i, j = divmod(i, 3)
            if n_row == 1:
                analyzer.plot_recall_auc(title, ax=axes[j])
            else:
                analyzer.plot_recall_auc(title, ax=axes[i, j])
        auc_vals.append(analyzer.recall_auc)
    if show_plot:
        plt.tight_layout()
    return analyzer
    

## Plot recall curves

In [ ]:
true_score_file = ... # Ground truth scores.
top_m = 20000
auc_n = 1000

with open(true_score_file, 'rb') as f:
    true_scores = pickle.load(f)

f_names = [ # SASS query score files.
    ...
]
freq = plot_recall(top_n=auc_n, true_scores=true_scores, f_names=f_names, is_score=True,
                   exp_product_ns=[top_m]*len(f_names))